# Define Functional Interface to Buidl Python Library to Create BIP 322 Signatures

## Required Interface

* Hash message
    * Message -> Tagged Hash of message
    * Note: Must handle legacy (has diff code in btc P.R.)
* Create BIP322Txs
  * Inputs (btc_address, message, signature (opt)
  * Output to_spend and to_sign Txs (Unsigned)
* Sign Message
    * Inputs (privkey, message)
    * Output signature
* Verify Message
    * Inputs (address, signature, message)
    * Output MessageVerificationResult

## Imports



In [1]:
from buidl.tx import Tx, TxIn, TxOut
from buidl.script import Script,P2WPKHScriptPubKey
from buidl.helper import big_endian_to_int, base64_decode, base64_encode, str_to_bytes

from buidl.script import address_to_script_pubkey
from buidl.hash import tagged_hash
from buidl.witness import Witness

from buidl.ecc import PrivateKey

import io


## Signature Formats

In [24]:
from enum import Enum

class MessageSignatureFormat(Enum):
    LEGACY = 0
    SIMPLE = 1
    FULL = 2
    

## Hash Message

In [36]:
def hash_message(message: str, format: MessageSignatureFormat = None):
    # TODO: Legacy hash
    
    if format == MessageSignatureFormat.LEGACY:
        raise NotImplementedError("Legacy hashing not yet implemented")
    
    b_msg = str_to_bytes(message)
    # Byte array of message hash
    # The tag defined in BIP0322 that should be used
    tag = b"BIP0322-signed-message"

    message_hash = tagged_hash(tag,b_msg)
    
    return message_hash

In [37]:
hash_message("somemsg", MessageSignatureFormat.LEGACY)

NotImplementedError: Legacy hashing not yet implemented

## Create BIP322Txs

In [4]:
# def create_bip322_txs(btc_address, message, sig_bytes=None):
    
#     to_spend = create_to_spend_tx(btc_address, message)
#     to_sign = create_to_sign_tx(to_spend.id(), sig_bytes)



def create_to_spend_tx(script_pubkey, message):
    # Not a valid Tx hash. Will never be spendable on any BTC network.
    prevout_hash = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
    # prevout.n
    prevout_index = big_endian_to_int(bytes.fromhex('FFFFFFFF'))
    
    sequence = 0


    message_hash = hash_message(message)

    # Note BIP322 to_spend scriptSig commands = [0, 32, message_hash]
    # PUSH32 is implied and added by the size of the message added to the stack
    commands = [0, message_hash]
    script_sig = Script(commands)
    # Create Tx Input
    tx_in = TxIn(prevout_hash,prevout_index,script_sig,sequence)
    
    # Value of tx output
    value = 0



    tx_out = TxOut(value,script_pubkey)
    
    # create transaction
    version=0
    tx_inputs = [tx_in]
    tx_outputs = [tx_out]
    locktime=0
    network="mainnet"

    # Could be false, but using a segwit address. I think this is the "Simple Signature" in BIP-0322
    segwit=True

    return Tx(version,tx_inputs,tx_outputs,locktime,network,segwit)




In [5]:
def create_to_sign_tx(to_spend_tx_hash, sig_bytes=None):
    to_sign = None
    if (sig_bytes and is_full_signature(sig_bytes)):
        
        sig_stream = io.BytesIO(decoded_test_sig)
        to_sign = Tx.parse(sig_stream)
        
        if (len(to_sign.tx_inputs) > 1):
            
            raise NotImplemented("Not yet implemented proof of funds yet")
        elif (len(to_sign.tx_inputs) == 0):
            raise ValueError("No transaction input")
        elif (to_sign.tx_inputs[0].prev_tx != to_spend_txid):
            raise ValueError("The to_sign transaction input's prevtx id does not equal the calculated to_spend transaction id")
        elif (len(to_sign.tx_outputs) != 1):
            raise ValueError("to_sign does not have a single TxOutput")
        elif (to_sign.tx_outputs[0].amount != 0):
            raise ValueError("Value is Non 0", to_sign.tx_outputs[0].amount)
        elif(to_sign.script_pubkey.commands != [106]):
            raise ValueError("ScriptPubKey incorrect", to_sign.script_pubkey)
            
    else:
        # signature is either None or an encoded witness stack
        
        # Identifies the index of the output from the virtual to_spend tx to be "spent"
        prevout_index = 0

        sequence = 0

        # TxInput identifies the output from to_spend
        tx_input = TxIn(to_spend_tx_hash,prevout_index,script_sig=None,sequence=sequence)
        

    
        value = 0
        # OP Code 106 for OP_RETURN
        commands = [106]
        scriptPubKey = Script(commands)

        tx_output = TxOut(value,scriptPubKey)
        
        version=0
        tx_inputs = [tx_input]
        tx_outputs = [tx_output]
        locktime=0
        network="mainnet"
        # Could be false, but using a segwit address. I think this is the "Simple Signature" in BIP-0322
        segwit=True
    
        # create unsigned to_sign transaction

        to_sign_tx = Tx(version,tx_inputs,tx_outputs,locktime,network,segwit)
        
        if sig_bytes:
            try:
                stream = io.BytesIO(sig_bytes)
                witness = Witness.parse(stream)
                # Set the witness on the to_sign tx input
                to_sign_tx.tx_ins[0].witness = witness
            except:
                # TODO: Fall back to legacy ...
                raise ValueError("Signature is neither a witness or full transaction")
                
        return to_sign_tx
            
        
        
        
        
        
        


In [6]:
# Test is sig_bytes can be decoded to a transaction
def is_full_signature(sig_bytes):
    try:
        sig_stream = io.BytesIO(decoded_test_sig)

        Tx.parse(sig_stream)
    # TODO: more specific exception handling
    except:
        return False
    return True
        

## Sign Message

Bitcoin core code as a scriptpubkeyman.cpp file that manages key pairs associated with a specific script I believe. This is not replicated in the buidl-python library

In [26]:
# Bitcoin doesn't expose this interface apart from for legacy signatures. 
# It does something with a abstract wallet interface which seems be be provided somehow ...
# I think this is partly about secure KMS
# 

# TODO: sign_message should take in a script rather than a private key? Or at least a script as well as a private key.
def sign_message(format: MessageSignatureFormat, private_key: PrivateKey, message: str):
    
    if (format != MessageSignatureFormat.LEGACY) {
        return sign_message_bip322(format,private_key_message)
    }

    # TODO: Handle legacy signing using p2pkh script
    
    
    

def sign_message_bip322(format: MessageSignatureFormat, private_key: PrivateKey, message: str):
    
    assert(format != MessageSignatureFormat.LEGACY)
    
    # How do we know which type of address to generate from the private key? Is this where the wallet comes in?
    
    # TODO: Not always going to be a p2wpkh address?
    
    # Generate a pay-to-witness-public-key-hash address
    p2wpkh_address = private_key.point.p2wpkh_address(network="mainnet")
    
    # Convert address to script_pubkey
    script_pubkey = address_to_script_pubkey(p2wpkh_address)
    
    to_spend = create_to_spend_tx(script_pubkey, message)
    to_sign = create_to_sign_tx(to_spend.hash(), None)
    
    to_sign.tx_ins[0]._script_pubkey = to_spend.tx_outs[0].script_pubkey
    to_sign.tx_ins[0]._value = to_spend.tx_outs[0].amount
    
    sig_ok = to_sign.sign_input(0, private_key)
    
    
    # Force the format to FULL, if this turned out to be a legacy format (p2pkh) signature
    if (len(to_sign.tx_ins[0].scriptSig) > 0 || len(to_sign.tx_ins[0].witness.items) == 0):
        format = MessageSignatureFormat.FULL
    
    
    if (!sig_ok):
        raise RuntimeError("Unable to sign message")
    
    if (format ==  MessageSignatureFormat.SIMPLE):
            return base64_encode(to_sign.serialize_witness())

    else:
        return base64_encode(to_sign.serialize())
    
    

    

In [27]:
test = "Hello World"

str_to_bytes(test)

b'Hello World'

In [29]:
compressed_wif_private_key = 'L3VFeEujGtevx9w18HD1fhRbCH67Az2dpCymeRE1SoPK6XQtaN2k'
private_key = PrivateKey.parse(compressed_wif_private_key)

message = "Hello World"


bip322_sig = sign_message(MessageSignatureFormat.SIMPLE,private_key, message)
print(bip322_sig)

AkgwRQIhAOzyynlqt93lOKJr+wmmxIens//zPzl9tqIOua93wO6MAiBi5n5EyAcPScOjf1lAqIUIQtr3zKNeavYabHyR8eGhowEhAsfxIAMZZEKUPYWI4BruhAQjzFT8FSFSajuFwrDL1Yhy


## Verify Message

In [16]:
def verify_message(address, signature, message):
    
    sig_bytes = base64_decode(signature)
    
    script_pubkey = address_to_script_pubkey(address)
    
    to_spend = create_to_spend_tx(script_pubkey, message)
    
    to_sign = create_to_sign_tx(to_spend.hash(), sig_bytes)
    
    to_sign.tx_ins[0]._script_pubkey = to_spend.tx_outs[0].script_pubkey
    to_sign.tx_ins[0]._value = to_spend.tx_outs[0].amount
    
    return to_sign.verify_input(0)
    
    

SyntaxError: invalid syntax (4228999517.py, line 1)

In [11]:

message = "Hello World"
address = "bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l"
test_sig = "AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI="


verify_message(address, test_sig, message)

True

In [12]:
verify_message(address, bip322_sig, message)

True